<a href="https://colab.research.google.com/github/Riverag0011/ADS509-Text-Mining/blob/main/Assignment_1_Lyrics_Scrape_GR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Lyrics Scraping

Name: Gabi Rivera \
Course: ADS509 \
Date: 09Sep2024 \

# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull.

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.*


# Importing Libraries

In [1]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [2]:
# Use this cell for any import statements you add
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.

In [3]:
artists = {'mileycyrus':"https://www.azlyrics.com/m/mileycyrus.html",
           'adele':"https://www.azlyrics.com/a/adele.html"}
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).)

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: It is allowed. We are requesting for artist_page. Disallowed terms are /lyricsdb/ and /song/.


In [4]:
# Dictionary to hold links to lyrics pages
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    try:
        # Request the page and sleep to avoid getting blocked
        r = requests.get(artist_page)
        time.sleep(5 + 10 * random.random())

        # Check if the request was successful
        if r.status_code == 200:
            print(f"Successfully retrieved page for {artist} (status code {r.status_code})")

            # Parse the page content
            soup = BeautifulSoup(r.text, 'html.parser')

            # Extract links to lyrics pages
            for a in soup.find_all('a', href=True):
                href = a['href']
                if href.startswith('/lyrics') and href.endswith('.html'):
                    full_url = f"https://www.azlyrics.com{href}"
                    lyrics_pages[artist].append(full_url)

            if not lyrics_pages[artist]:
                print(f"No lyrics links found for {artist}.")
        else:
            print(f"Failed to retrieve page for {artist} with status code {r.status_code}")
    except Exception as e:
        print(f"An error occurred for {artist}: {e}")

Successfully retrieved page for mileycyrus (status code 200)
Successfully retrieved page for adele (status code 200)


Let's make sure we have enough lyrics pages to scrape.

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20)

In [6]:
# Let's see how long it's going to take to pull these lyrics
# if we're waiting `5 + 10*random.random()` seconds
for artist, links in lyrics_pages.items() :
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For mileycyrus we have 174.
The full pull will take for this artist will take 0.48 hours.
For adele we have 71.
The full pull will take for this artist will take 0.2 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

1. Create an empty folder in our repo called "lyrics".
1. Iterate over the artists in `lyrics_pages`.
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages.
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.


In [7]:
# Function to generate filename from URL
def generate_filename_from_url(link):
    if not link:
        return None

    # Drop the http or https and the html
    name = link.replace("https", "").replace("http", "")
    name = name.replace(".html", "")

    # Replace useless characters with UNDERSCORE
    name = name.replace("/lyrics/", "")
    name = name.replace("://", "").replace(".", "_").replace("/", "_")

    # Tack on .txt
    name = name + ".txt"

    return name

In [8]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") :
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [9]:
# Base URL for constructing links
url_stub = "https://www.azlyrics.com"

start = time.time()
total_pages = 0
max_songs_per_artist = 20

for artist, links in lyrics_pages.items():
    # Create a subfolder for the artist
    artist_folder = os.path.join("lyrics", artist)
    os.makedirs(artist_folder, exist_ok=True)

    # Limit the number of songs to 20
    song_count = 0

    for link in links:
        if song_count >= max_songs_per_artist:
            break

        # Request the lyrics page
        r = requests.get(link)
        time.sleep(5 + 10 * random.random())

        if r.status_code == 200:
            print(f"Successfully retrieved lyrics for {artist} from {link}")
            # Parse the lyrics page
            soup = BeautifulSoup(r.text, 'html.parser')

            # Extract the title and lyrics
            title_tag = soup.find('title')
            if title_tag:
                title = title_tag.text.split('Lyrics')[0].strip()
            else:
                title = "Unknown_Title"

            lyrics_tag = soup.find('div', class_='col-xs-12 col-lg-8 text-center')
            if lyrics_tag:
                lyrics = lyrics_tag.get_text(separator='\n').strip()
            else:
                lyrics = "Lyrics not found."

            # Generate filename and write the lyrics to a file
            filename = generate_filename_from_url(link)
            file_path = os.path.join(artist_folder, filename)

            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(f"{title}\n\n{lyrics}")

            total_pages += 1
            song_count += 1

# Print total run time
print(f"Total pages scraped: {total_pages}")
print(f"Total run time was {round((time.time() - start) / 3600, 2)} hours.")

Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/breakout.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/7things.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/thedriveway.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/girlsjustwannahavefun.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/fullcircle.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/flyonthewall.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/bottomoftheocean.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/wakeupamerica.html
Successfully retrieved lyrics for mileycyrus from https://www.azlyrics.com/lyrics/mileycyrus/fourwalls.html
Su

---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [10]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text):
    return re.findall(r'\w+', text.lower())

## Checking Lyrics

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work.

In [11]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders :
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files :
        with open("lyrics/" + artist + "/" + f_name) as infile :
            artist_words.extend(words(infile.read()))


    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For mileycyrus we have 20 files.
For mileycyrus we have roughly 12171 words, 1559 are unique.
For adele we have 20 files.
For adele we have roughly 13171 words, 1731 are unique.
